In [ ]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

from keras_facenet import FaceNet
from keras.preprocessing.image import load_img, img_to_array

In [2]:
df = pd.read_csv('celeb_embeddings.csv')
df.sample(3)

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,target
321,-0.003574,0.026938,-0.055958,0.025274,0.060849,-0.015289,0.008360,0.173274,-0.009592,0.034589,...,-0.052019,0.024860,0.014458,0.006939,0.056616,0.014577,0.031004,0.007175,-0.037235,1
81,0.040185,0.018409,0.026871,-0.074859,-0.016567,-0.054832,0.025275,-0.028597,0.106934,-0.036243,...,0.033754,0.030762,-0.003985,-0.065546,-0.033495,-0.019550,-0.002377,-0.033387,-0.086241,1
957,0.057349,0.067941,0.001601,0.002162,0.094676,-0.035296,0.024454,0.070425,0.054991,-0.000935,...,-0.060910,0.002720,0.016191,0.035951,0.022001,0.014406,-0.015582,0.006471,-0.083581,0


In [3]:
# Separate the features and the target
X = df.drop('target', axis=1)
y = df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# Create and train the classifier
clf = SVC(C=1.0, kernel='rbf', gamma='scale')
clf.fit(X_train, y_train)

In [5]:
# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Print a classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        57
           1       1.00      0.98      0.99        53

    accuracy                           0.99       110
   macro avg       0.99      0.99      0.99       110
weighted avg       0.99      0.99      0.99       110



In [9]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[57  0]
 [ 1 52]]


In [6]:
# Retrain the classifier with the entire dataset and save it
clf = SVC(C=1.0, kernel='rbf', gamma='scale')
clf.fit(X, y)

# Save the classifier
with open('trained_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [ ]:
####### tests #######

from process_pipeline import ImageProcessor
processor = ImageProcessor()

# ##### Process the test images #####
image_test_01 = 'test_images/01_buscemi.jpg'
image_test_02 = 'test_images/02_pitt.jpg'

pitt_test = processor.process_image(image_test_01)
devito_test = processor.process_image(image_test_02)

In [8]:
# ##### Save the processed images #####
output_directory = "test_images"
filename = "pitt_test_processed.jpg"
processor.save_image(pitt_test, output_directory, filename)
pitt_test_processed_path = output_directory + "\\" + filename

filename = "devito_test_processed.jpg"
processor.save_image(devito_test, output_directory, filename)
devito_test_processed_path = output_directory + "\\" + filename

Aligned face has been saved in the 'test_images' directory as pitt_test_processed.jpg.
Path to saved new image: test_images\pitt_test_processed.jpg
Aligned face has been saved in the 'test_images' directory as devito_test_processed.jpg.
Path to saved new image: test_images\devito_test_processed.jpg


In [9]:
# Define a function to extract embeddings from an image
def extract_embedding(image_path, model):
    # Load and preprocess the image
    image = load_img(image_path, target_size=(160, 160), color_mode='rgb')
    image = img_to_array(image)
    # Extract the embedding using the model
    embedding = model.embeddings(np.array([image]))[0]
    # Convert the embedding to a DataFrame
    df = pd.DataFrame([embedding])

    return df

In [10]:
embedder = FaceNet()

# Extract the embedding using the Function
pitt_is_allowed = extract_embedding(pitt_test_processed_path, embedder)
devito_not_allowed = extract_embedding(devito_test_processed_path, embedder)

1/1 [==============================] - 0s 42ms/step


In [11]:
feature_names = X_train.columns.tolist()

# save the feature names in a .txt file
with open('feature_names.txt', 'w') as f:
    f.write(f"features_names = {feature_names}")

In [12]:
# transform the embedding in a dataframe
pitt_is_allowed = pd.DataFrame(pitt_is_allowed)
pitt_is_allowed.columns = feature_names

devito_not_allowed = pd.DataFrame(devito_not_allowed)
devito_not_allowed.columns = feature_names

In [13]:
## if filename.startswith('Faces_Dataset_processed/allowed'):
##     target = 1
## elif filename.startswith('Faces_Dataset_processed/not_allowed'):
##     target = 0

In [14]:
top_model = pickle.load(open('trained_classifier.pkl', 'rb'))

In [15]:
# Prediction pitt_is_allowed
predict_test = top_model.predict(pitt_is_allowed)

if predict_test[0] == 1:
    print("Allowed")
else:
    print("Not Allowed")

Allowed


In [16]:
# Prediction devito_not_allowed
predict_test = top_model.predict(devito_not_allowed)

if predict_test[0] == 1:
    print("Allowed")
else:
    print("Not Allowed")

Not Allowed
